<a href="https://colab.research.google.com/github/matyusha/MultMatrix/blob/main/MultMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install pycuda

In [50]:
import pycuda.autoinit
from pycuda.tools import make_default_context
make_default_context().get_device().name()

'Tesla T4'

In [52]:
import numpy as np
from numpy import linalg as la
from pycuda import driver, compiler, gpuarray, tools
from time import time

for i in range(0, 5):
  
  MATRIX_SIZE = 128*2**i
  print("MATRIX_SIZE", MATRIX_SIZE)

  def matmul(a_gpu,b_gpu,MATRIX_SIZE=MATRIX_SIZE):
    kernel_code_template = """
    __global__ void MatrixMulKernel(float *A, float *B, float *C)
    {

      const uint wA = %(MATRIX_SIZE)s;
      const uint wB = %(MATRIX_SIZE)s;

      const uint bx = blockIdx.x;
      const uint by = blockIdx.y;

      const uint tx = threadIdx.x;
      const uint ty = threadIdx.y;

      const uint aBegin = wA * %(BLOCK_SIZE)s * by;
      const uint aEnd = aBegin + wA - 1;
      const uint aStep = %(BLOCK_SIZE)s;

      const uint bBegin = %(BLOCK_SIZE)s * bx;
      const uint bStep = %(BLOCK_SIZE)s * wB;

      float Csub = 0;
      for (int a = aBegin, b = bBegin;
           a <= aEnd;
           a += aStep, b += bStep)
        {
          __shared__ float As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
          __shared__ float Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

          As[ty][tx] = A[a + wA * ty + tx];
          Bs[ty][tx] = B[b + wB * ty + tx];

          __syncthreads();

          for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
            Csub += As[ty][k] * Bs[k][tx];

          __syncthreads();
        }

      const uint c = wB * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
      C[c + wB * ty + tx] = Csub;

    }
    """

    TILE_SIZE = 32
    BLOCK_SIZE = TILE_SIZE

    kernel_code = kernel_code_template % {
        'MATRIX_SIZE': MATRIX_SIZE,
        'BLOCK_SIZE': BLOCK_SIZE,
        }

    mod = compiler.SourceModule(kernel_code)
    c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

    matrixmul = mod.get_function("MatrixMulKernel")

    matrixmul(
        a_gpu, b_gpu,
        c_gpu,
        grid = (MATRIX_SIZE // TILE_SIZE, MATRIX_SIZE // TILE_SIZE),
        block = (TILE_SIZE, TILE_SIZE, 1),
        )

    return c_gpu

  a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
  b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

  st = time()
  c_cpu = np.dot(a_cpu, b_cpu)
  cpu_time = time() - st

  st = time()
  a_gpu = gpuarray.to_gpu(a_cpu)
  b_gpu = gpuarray.to_gpu(b_cpu)
  c_gpu = matmul(a_gpu,b_gpu).get()
  gpu_time = time() - st

  print("TIME ON CPU: {:.6f}".format(cpu_time))
  print("TIME ON GPU: {:.6f}".format(gpu_time))
  print("SPEEDUP {:.3f}".format(cpu_time/gpu_time)) 
  print("CPU-GPU difference: {:.6f}".format(np.max(c_cpu - c_gpu)))
  print()
  print("-" * 40)

MATRIX_SIZE 128
TIME ON CPU: 0.000235
TIME ON GPU: 0.003399
SPEEDUP 0.069
CPU-GPU difference: 0.000000

----------------------------------------
MATRIX_SIZE 256
TIME ON CPU: 0.000738
TIME ON GPU: 0.002383
SPEEDUP 0.310
CPU-GPU difference: 0.000000

----------------------------------------
MATRIX_SIZE 512
TIME ON CPU: 0.004339
TIME ON GPU: 0.003542
SPEEDUP 1.225
CPU-GPU difference: 0.000084

----------------------------------------
MATRIX_SIZE 1024
TIME ON CPU: 0.037209
TIME ON GPU: 0.011404
SPEEDUP 3.263
CPU-GPU difference: 0.000183

----------------------------------------
MATRIX_SIZE 2048
TIME ON CPU: 0.270370
TIME ON GPU: 0.056751
SPEEDUP 4.764
CPU-GPU difference: 0.000549

----------------------------------------
